In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import constants, units
from astropy.coordinates import Angle
from scipy.optimize import curve_fit
import pandas as pd
import os

In [2]:
rootdir = '/Users/thepoetoftwilight/Documents/CUBS/Data/PG1522+101/'

Load in the photometry catalogs - first MUSE

In [3]:
MUSE_df = pd.read_csv(rootdir+'MUSE/pseudo_gri_photometry.dat')

In [4]:
MUSE_df

,ID,RA,Dec,x,y,pseudo_g_mag,pseudo_r_mag,pseudo_i_mag,z
0,1,231.107296,9.967454,76.8023,29.5341,23.22,21.99,21.41,0.5356
1,2,231.105051,9.967208,116.6069,25.0939,23.43,22.76,22.62,0.4581
2,3,231.096042,9.966748,276.3172,16.8158,25.12,24.19,23.21,0.9576
3,5,231.103970,9.966703,135.7645,16.0164,25.63,24.67,24.01,0.8217
4,18,231.103628,9.981773,141.8330,287.2714,19.83,19.34,19.14,0.0000
...,...,...,...,...,...,...,...,...,...
75,109,231.109148,9.967285,43.9638,26.4939,27.75,27.10,26.66,-1.0000
76,114,231.106211,9.982972,96.0353,308.8459,24.74,23.53,22.84,0.0000
77,115,231.107990,9.982978,64.5125,308.9630,25.49,25.19,24.46,-1.0000
78,122,231.101477,9.982300,179.9579,296.7565,25.21,24.75,24.70,0.0966


Next, HST F160W

In [5]:
F160W_df = pd.read_csv(rootdir+'HST_images/f160w_photometry.dat')

In [6]:
F160W_df

,ID,RA,Dec,x,y,f160w_mag
0,1,231.089271,9.957233,1026.9171,157.8502,20.8196
1,2,231.090045,9.956782,1010.1237,139.3674,21.3203
2,3,231.090317,9.953885,1026.9756,59.1561,23.0746
3,4,231.091208,9.952799,1012.3724,22.6274,24.3964
4,5,231.088950,9.954643,1056.9227,90.7578,18.7594
...,...,...,...,...,...,...
1077,1078,231.097474,9.988867,546.6360,942.3546,25.0309
1078,1079,231.103001,9.990440,387.0121,939.5074,24.7557
1079,1080,231.109894,9.992215,189.4819,930.9360,22.6273
1080,1081,231.100346,9.989033,469.0965,923.3248,21.4978


Finally, HST F140W

In [7]:
F140W_df = pd.read_csv(rootdir+'HST_images/f140w_photometry.dat')

In [8]:
F140W_df

,ID,RA,Dec,x,y,f140w_mag
0,1,231.087362,9.960261,1042.0133,252.2663,21.0268
1,2,231.087681,9.959454,1040.2750,227.9585,19.7528
2,3,231.117436,9.965145,203.9400,137.5319,21.1236
3,4,231.118888,9.964064,174.4381,96.6139,19.4755
4,5,231.091174,9.953225,999.4924,31.9852,24.3223
...,...,...,...,...,...,...
1109,1110,231.092237,9.987699,684.4825,949.7799,24.0699
1110,1111,231.115955,9.960226,284.1344,17.4685,25.1467
1111,1112,231.099320,9.989777,479.3973,947.7029,23.6488
1112,1113,231.087397,9.986150,825.7208,947.7474,25.2002


First, we should join F140W (the largest catalog) with F160W (the next largest catalog). We'll compute the angular separation between all possible pairs of objects using the formula. Below, we present the formalism -

Point 1: ($\alpha_1, \delta_1$), Point 2: ($\alpha_2, \delta_2$)

Angular separation $\phi$ between unit vectors -

$$\cos(\phi) = \hat{r_1} \cdot \hat{r_2}$$

$$\Rightarrow \cos(\phi) = \cos(\delta_1) \cos(\delta_2) \cos(\alpha_1) \cos(\alpha_2) + \cos(\delta_1) \cos(\delta_2) \sin(\alpha_1) \sin(\alpha_2) + \sin(\delta_1) \sin(\delta_2)$$

$$\boxed{\phi = \arccos[ \cos(\delta_1) \cos(\delta_2) \cos(\alpha_1) \cos(\alpha_2) + \cos(\delta_1) \cos(\delta_2) \sin(\alpha_1) \sin(\alpha_2) + \sin(\delta_1) \sin(\delta_2)]}$$

In [9]:
def calc_phi(alpha_1, delta_1, alpha_2, delta_2):
    
    cos_phi = np.dot([np.cos(delta_1)*np.cos(alpha_1), np.cos(delta_1)*np.sin(alpha_1), np.sin(delta_1)],
                      [np.cos(delta_2)*np.cos(alpha_2), np.cos(delta_2)*np.sin(alpha_2), np.sin(delta_2)])
    
    phi = np.arccos(cos_phi)
    
    return phi

We'll define a function to create a grid of angular separations

In [10]:
def calc_phi_grid(df_1, df_2):
    
    # Get RAs and Decs from each dataframe
    df_1_RA = df_1['RA']
    df_1_Dec = df_1['Dec']
    
    df_2_RA = df_2['RA']
    df_2_Dec = df_2['Dec']
    
    # Cast each of them in degrees
    df_1_RA_deg = [a*units.deg for a in df_1_RA]
    df_1_Dec_deg = [a*units.deg for a in df_1_Dec]
    
    df_2_RA_deg = [a*units.deg for a in df_2_RA]
    df_2_Dec_deg = [a*units.deg for a in df_2_Dec]  
    
    # Convert to radians now
    df_1_RA_rad = [a.to(units.rad) for a in df_1_RA_deg]
    df_1_Dec_rad = [a.to(units.rad) for a in df_1_Dec_deg]

    df_2_RA_rad = [a.to(units.rad) for a in df_2_RA_deg]
    df_2_Dec_rad = [a.to(units.rad) for a in df_2_Dec_deg]
    
    # Define a grid of phi values
    phi_grid = np.zeros((len(df_1), len(df_2)))
    
    for i in range(len(df_1_RA_rad)):
        for j in range(len(df_2_RA_rad)):
            
            # Isolate the relevant angles
            df_1_RA = df_1_RA_rad[i]
            df_1_Dec = df_1_Dec_rad[i]

            df_2_RA = df_2_RA_rad[j]
            df_2_Dec = df_2_Dec_rad[j]

            phi = (calc_phi(df_1_RA, df_1_Dec, df_2_RA, df_2_Dec)*units.radian).to(units.arcsecond).value
            
            phi_grid[i,j] = phi
            
    return phi_grid

In [11]:
def calc_match_indices(phi_grid, phi_thresh=1):
    
    # Record the indices of closely separated objects, and also their separation
    match_idx_1 = []
    match_idx_2 = []
    match_phi = []
    
    for i in range(phi_grid.shape[0]):
        for j in range(phi_grid.shape[1]):
            
            phi = phi_grid[i,j]
            
            if(phi<=phi_thresh):
                match_idx_1.append(i)
                match_idx_2.append(j)
                match_phi.append(phi)
                
    return np.array(match_idx_1), np.array(match_idx_2), np.array(match_phi)

In [ ]:
phi_grid_HST = calc_phi_grid(F140W_df, F160W_df)

In [ ]:
# Get the matching indices for the two HST catalogs
F140W_df_idx, F160W_df_idx, phi_match_HST = calc_match_indices(phi_grid_HST, phi_thresh=.3)

In [ ]:
# Now create a copy of the F140W catalog
master_df = F140W_df.copy()

In [ ]:
master_df['f160w_mag'] = ['-1.0' for i in range(len(master_df))]
master_df['phi_HST'] = ['-1.0' for i in range(len(master_df))]

In [ ]:
# In index locations of the F140W catalog where an F160W object is "nearby"
# Enter the F160W magnitudes, and also the separation of the object
master_df.loc[F140W_df_idx, 'f160w_mag'] = np.array(F160W_df.loc[F160W_df_idx,'f160w_mag'])
master_df.loc[F140W_df_idx, 'phi_HST'] = phi_match_HST

In [ ]:
master_df.loc[F140W_df_idx]

Now, merge in the MUSE catalog

In [ ]:
phi_grid_MUSE = calc_phi_grid(master_df, MUSE_df)

In [ ]:
master_df_idx, MUSE_df_idx, phi_match_MUSE = calc_match_indices(phi_grid_MUSE, phi_thresh=.3)

In [ ]:
master_df_final = master_df.copy()

In [ ]:
master_df_final['pseudo_g_mag'] = ['-1.0' for i in range(len(master_df))]
master_df_final['pseudo_r_mag'] = ['-1.0' for i in range(len(master_df))]
master_df_final['pseudo_i_mag'] = ['-1.0' for i in range(len(master_df))]
master_df_final['phi_MUSE'] = ['-1.0' for i in range(len(master_df))]
master_df_final['z'] = ['-1.0' for i in range(len(master_df))]

In [ ]:
master_df_final.loc[master_df_idx, 'pseudo_g_mag'] = np.array(MUSE_df.loc[MUSE_df_idx, 'pseudo_g_mag'])
master_df_final.loc[master_df_idx, 'pseudo_r_mag'] = np.array(MUSE_df.loc[MUSE_df_idx, 'pseudo_r_mag'])
master_df_final.loc[master_df_idx, 'pseudo_i_mag'] = np.array(MUSE_df.loc[MUSE_df_idx, 'pseudo_i_mag'])
master_df_final.loc[master_df_idx, 'phi_MUSE'] = phi_match_MUSE
master_df_final.loc[master_df_idx, 'z'] = np.array(MUSE_df.loc[MUSE_df_idx, 'z'])

Here is the final catalog

In [ ]:
master_df_final

In [ ]:
# Here are the overlapping entries with MUSE
# Why are there more objects than the MUSE catalog?
master_df_final.loc[master_df_idx]

Compile the final catalog

In [ ]:
gal_ra_arr = np.array(master_df_final['RA'])
gal_dec_arr = np.array(master_df_final['Dec'])
gal_f160w_mag_arr = np.array(master_df_final['f160w_mag'])
gal_f140w_mag_arr = np.array(master_df_final['f140w_mag'])
gal_pseudo_g_mag_arr = np.array(master_df_final['pseudo_g_mag'])
gal_pseudo_r_mag_arr = np.array(master_df_final['pseudo_r_mag'])
gal_pseudo_i_mag_arr = np.array(master_df_final['pseudo_i_mag'])
gal_z_arr = np.array(master_df_final['z'])

In [ ]:
with open(rootdir+'ldss_photometry.dat', 'w') as f:

    f.write('RA,Dec,f160w_mag,f140w_mag,pseudo_g_mag,pseudo_r_mag,pseudo_i_mag,z')

    for i in range(len(gal_ra_arr)):

        f.write('\n'+str(gal_ra_arr[i])+','+
             str(gal_dec_arr[i])+','+
             str(gal_f160w_mag_arr[i])+','+
             str(gal_f140w_mag_arr[i])+','+
             str(gal_pseudo_g_mag_arr[i])+','+
             str(gal_pseudo_r_mag_arr[i])+','+
             str(gal_pseudo_i_mag_arr[i])+','+
             str(gal_z_arr[i]))

Also write the subset of MUSE objects into a separate catalog

In [ ]:
master_df_overlap = master_df_final.loc[master_df_idx]

In [ ]:
gal_ra_overlap_arr = np.array(master_df_overlap['RA'])
gal_dec_overlap_arr = np.array(master_df_overlap['Dec'])
gal_f160w_mag_overlap_arr = np.array(master_df_overlap['f160w_mag'])
gal_f140w_mag_overlap_arr = np.array(master_df_overlap['f140w_mag'])
gal_pseudo_g_mag_overlap_arr = np.array(master_df_overlap['pseudo_g_mag'])
gal_pseudo_r_mag_overlap_arr = np.array(master_df_overlap['pseudo_r_mag'])
gal_pseudo_i_mag_overlap_arr = np.array(master_df_overlap['pseudo_i_mag'])
gal_z_overlap_arr = np.array(master_df_overlap['z'])

In [ ]:
with open(rootdir+'ldss_photometry_subset.dat', 'w') as f:

    f.write('RA,Dec,f160w_mag,f140w_mag,pseudo_g_mag,pseudo_r_mag,pseudo_i_mag,z')

    for i in range(len(gal_ra_overlap_arr)):

        f.write('\n'+str(gal_ra_overlap_arr[i])+','+
             str(gal_dec_overlap_arr[i])+','+
             str(gal_f160w_mag_overlap_arr[i])+','+
             str(gal_f140w_mag_overlap_arr[i])+','+
             str(gal_pseudo_g_mag_overlap_arr[i])+','+
             str(gal_pseudo_r_mag_overlap_arr[i])+','+
             str(gal_pseudo_i_mag_overlap_arr[i])+','+
             str(gal_z_overlap_arr[i]))